In [ ]:
from dask.distributed import Client

In [ ]:
#$ dask-scheduler

#$ dask-worker 127.0.0.1:8786
#$ dask-worker 127.0.0.1:8786
#$ dask-worker 127.0.0.1:8786


client = Client() # '127.0.0.1:8786'
client

In [ ]:
from dask import dataframe as dd
import pandas

### Get Data

In [ ]:
# fpath = '../data/cleaned/block_*.csv'
fpath = '../data/cleaned/block_0.csv'

#'tstp': 'parse_dates', # 'M8[us]',  # pandas.Timestamp, 

# TODO: ???
id_key = 'LCLid'
usage_key = 'energy(kWh/hh)'
date_key = 'tstp'

# Schema     
schema = {
    'LCLid': 'category',
    'energy(kWh/hh)': 'float64',
}
date_cols = ['tstp']

usage_data = dd.read_csv(fpath, dtype=schema, parse_dates=date_cols)

usage_data.head()

In [ ]:
if not usage_data[id_key].cat.known:  # Compute the MACs
    usage_data[id_key] = usage_data[id_key].cat.as_known()
    
usage_data[id_key].cat.categories

### Set index depending on operations

In [ ]:
# usage_data.set_index('LCLid')
usage_data.set_index(date_key)

In [ ]:
test_out = '../data/parquet/test_out'
usage_data.to_parquet(test_out, engine='pyarrow')



In [ ]:
def convert(x):
    if x == 'Null' or x == 'foo':
        return None
    else:
        return float(x.strip())

# series = df['energy(kWh/hh)'].apply(convert)

### Smoke test Dask

In [ ]:
usage_mean = usage_data[usage_key].mean()
usage_mean.compute()

### Split MACIDs

In [ ]:
# usage_data['LCLid'] = usage_data['LCLid'].astype('category')

In [ ]:
usage_data.describe().compute()

In [ ]:
usage_data[usage_data[id_key] == 'MAC000002'].describe().compute()

In [ ]:
usage_data[usage_data[id_key] == 'MAC000246'].describe().compute()

In [ ]:
usage_data[usage_key].mean().compute()

In [ ]:
usage_data.groupby(id_key)[usage_key].mean().compute()


In [ ]:
import pandas

In [ ]:
hhour_fname = 'block_1.csv'
hhour_fpath = f"{hhour_dir}/{hhour_fname}"

df = pandas.read_csv(hhour_fpath)
df.describe()

In [ ]:
df['energy(kWh/hh)'].mean()